# Desenvolvimento de um Corretor Ortográfico com NLTK

## Imports e leitura dos dados

In [9]:
import nltk

In [4]:
with open('./corretor-master/artigos.txt', 'r', encoding="utf8") as f:
    artigos = f.read()

artigos[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

## Testes

In [5]:
texto_exemplo = 'Olá, tudo bem?'

In [12]:
nltk.download('punkt')
palavras_separadas = nltk.tokenize.word_tokenize(texto_exemplo)
palavras_separadas

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Marcelo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


['Olá', ',', 'tudo', 'bem', '?']

In [18]:
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

separa_palavras(texto_exemplo)

['O', 'l', 'á', 't', 'u', 'd', 'o', 'b', 'e', 'm']

## Aplicando nos dados dos Artigos

### Separando as palavras dos artigos

In [20]:
lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)
len(lista_palavras)

403031

### Vamos descobrir a quantidade de tipos de palavras

Primeiro, devemos normalizar o texto, para que assim palavras maiúsculas e minúsculas sejam contadas como iguais.

In [29]:
def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)
lista_normalizada[:5]

['imagem', 'temos', 'a', 'seguinte', 'classe']

Agora, vamos utilizar o set(), já que ele não contém valores repetidos. Dessa forma, será possível saber quantos tipos de palavras (únicas) temos na nossa base de textos.

In [30]:
len(set(lista_normalizada))

18464

## Construção do Corretor

In [34]:
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

In [35]:
palavra_exemplo = 'lgica'

def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas)

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica', 'klgica', 'llgica', 'mlgica', 'nlgica', 'olgica', 'plgica', 'qlgica', 'rlgica', 'slgica', 'tlgica', 'ulgica', 'vlgica', 'wlgica', 'xlgica', 'ylgica', 'zlgica', 'àlgica', 'álgica', 'âlgica', 'ãlgica', 'èlgica', 'élgica', 'êlgica', 'ìlgica', 'ílgica', 'îlgica', 'òlgica', 'ólgica', 'ôlgica', 'õlgica', 'ùlgica', 'úlgica', 'ûlgica', 'çlgica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'làgica', 'lágica', 'lâgica', 'lãgica', 'lègica', 'légica', 'lêgica', 'lìgica', 'lígica', 'lîgica', 'lògica', 'lógica', 'lôgica', 'lõgica', 'lùgica', 'lúgica', 'lûgica', 'lçgica', 'lgaica', 'lgbica', 'lgcica', 'lgdica', 'lgeica', 'lgfica', 'lggica', 'lghica', 'lgiica', 'lgjica', 'lgkica', 'lglica',

### Construindo o validador da palavra correta

In [37]:
total_palavras = len(lista_normalizada)
frequencia = nltk.FreqDist(lista_normalizada)

In [44]:
def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras

print(probabilidade('logica'), probabilidade('lógica'))

0.0 0.00023819507680550628


In [42]:
def corretor(palavra_equivocada):
    palavras_geradas = gerador_palavras(palavra_equivocada)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [43]:
corretor(palavra_exemplo)

'lógica'

## Corrigindo outros tipos de erro

### Avaliando o corretor

In [55]:
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, 'r', encoding='utf8')
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

lista_teste = cria_dados_teste('./corretor-master/palavras.txt')
lista_teste

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm'),
 ('ele', 'eme'),
 ('fazer', 'èazer'),
 ('temos', 'temfs'),
 ('essa', 'eàssa'),
 ('quando', 'quaôdo'),
 ('vamos', 'vamvos'),
 ('sobre', 'hsobre'),
 ('java', 'sjava'),
 ('das', 'daõs'),
 ('agora', 'agorah'),
 ('está', 'eòtá'),
 ('cada', 'céda'),
 ('mesmo', 'zmesmo'),
 ('nos', 'noâ'),
 ('forma', 'fobma'),
 ('seja', 'sejéa'),
 ('então', 'enêão'),
 ('criar', 'èriar'),
 ('código', 'cóeigo'),
 ('caso', 'casío'),
 ('exemplo', 'áexemplo'),
 ('tem', 'tĩem'),
 ('usuário', 'usuárôio'),
 ('dados', 'dfados'),
 ('python', 'pgthon'),
 ('nossa', 'nossah'),
 ('além', 'alémè'),
 ('assim', 'asõim'),
 ('ter', 'teb'),
 ('até', 'atĩ'),
 ('bem', 'âem'),
 ('design', 'desigen'),
 ('trabalho', 'trabalàho'),
 ('foi', 'foo'),
 ('apenas', 'apenaũ'),
 ('empresa', 'empresà'),
 ('valor', 'valíor'),
 ('será', 'serr')

In [57]:
def avaliador(testes):
    numero_palavras = len(testes)
    acertou = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras')

avaliador(lista_teste)

1.08% de 186 palavras


### Deletando caracteres

In [59]:
def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

In [60]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    return palavras_geradas

palavra_exemplo = 'lóigica'
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas)

['alóigica', 'blóigica', 'clóigica', 'dlóigica', 'elóigica', 'flóigica', 'glóigica', 'hlóigica', 'ilóigica', 'jlóigica', 'klóigica', 'llóigica', 'mlóigica', 'nlóigica', 'olóigica', 'plóigica', 'qlóigica', 'rlóigica', 'slóigica', 'tlóigica', 'ulóigica', 'vlóigica', 'wlóigica', 'xlóigica', 'ylóigica', 'zlóigica', 'àlóigica', 'álóigica', 'âlóigica', 'ãlóigica', 'èlóigica', 'élóigica', 'êlóigica', 'ìlóigica', 'ílóigica', 'îlóigica', 'òlóigica', 'ólóigica', 'ôlóigica', 'õlóigica', 'ùlóigica', 'úlóigica', 'ûlóigica', 'çlóigica', 'laóigica', 'lbóigica', 'lcóigica', 'ldóigica', 'leóigica', 'lfóigica', 'lgóigica', 'lhóigica', 'lióigica', 'ljóigica', 'lkóigica', 'llóigica', 'lmóigica', 'lnóigica', 'loóigica', 'lpóigica', 'lqóigica', 'lróigica', 'lsóigica', 'ltóigica', 'luóigica', 'lvóigica', 'lwóigica', 'lxóigica', 'lyóigica', 'lzóigica', 'làóigica', 'láóigica', 'lâóigica', 'lãóigica', 'lèóigica', 'léóigica', 'lêóigica', 'lìóigica', 'líóigica', 'lîóigica', 'lòóigica', 'lóóigica', 'lôóigica', 'lõ

In [61]:
avaliador(lista_teste)

41.4% de 186 palavras


### Trocando letras

In [62]:
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras


def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras


def troca_letra(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras


def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_letra(fatias)
    return palavras_geradas

palavra_exemplo = 'lígica'
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas)

['alígica', 'blígica', 'clígica', 'dlígica', 'elígica', 'flígica', 'glígica', 'hlígica', 'ilígica', 'jlígica', 'klígica', 'llígica', 'mlígica', 'nlígica', 'olígica', 'plígica', 'qlígica', 'rlígica', 'slígica', 'tlígica', 'ulígica', 'vlígica', 'wlígica', 'xlígica', 'ylígica', 'zlígica', 'àlígica', 'álígica', 'âlígica', 'ãlígica', 'èlígica', 'élígica', 'êlígica', 'ìlígica', 'ílígica', 'îlígica', 'òlígica', 'ólígica', 'ôlígica', 'õlígica', 'ùlígica', 'úlígica', 'ûlígica', 'çlígica', 'laígica', 'lbígica', 'lcígica', 'ldígica', 'leígica', 'lfígica', 'lgígica', 'lhígica', 'liígica', 'ljígica', 'lkígica', 'llígica', 'lmígica', 'lnígica', 'loígica', 'lpígica', 'lqígica', 'lrígica', 'lsígica', 'ltígica', 'luígica', 'lvígica', 'lwígica', 'lxígica', 'lyígica', 'lzígica', 'làígica', 'láígica', 'lâígica', 'lãígica', 'lèígica', 'léígica', 'lêígica', 'lìígica', 'líígica', 'lîígica', 'lòígica', 'lóígica', 'lôígica', 'lõígica', 'lùígica', 'lúígica', 'lûígica', 'lçígica', 'líagica', 'líbgica', 'lícgica'

### Inversão de letras

In [67]:
def inverte_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras


def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras


def troca_letra(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras


def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_letra(fatias)
    palavras_geradas += inverte_letra(fatias)
    return palavras_geradas

palavra_exemplo = 'lógiac'
palavras_geradas = gerador_palavras(palavra_exemplo)
print(palavras_geradas, 'lógica' in palavras_geradas)

['alógiac', 'blógiac', 'clógiac', 'dlógiac', 'elógiac', 'flógiac', 'glógiac', 'hlógiac', 'ilógiac', 'jlógiac', 'klógiac', 'llógiac', 'mlógiac', 'nlógiac', 'ológiac', 'plógiac', 'qlógiac', 'rlógiac', 'slógiac', 'tlógiac', 'ulógiac', 'vlógiac', 'wlógiac', 'xlógiac', 'ylógiac', 'zlógiac', 'àlógiac', 'álógiac', 'âlógiac', 'ãlógiac', 'èlógiac', 'élógiac', 'êlógiac', 'ìlógiac', 'ílógiac', 'îlógiac', 'òlógiac', 'ólógiac', 'ôlógiac', 'õlógiac', 'ùlógiac', 'úlógiac', 'ûlógiac', 'çlógiac', 'laógiac', 'lbógiac', 'lcógiac', 'ldógiac', 'leógiac', 'lfógiac', 'lgógiac', 'lhógiac', 'liógiac', 'ljógiac', 'lkógiac', 'llógiac', 'lmógiac', 'lnógiac', 'loógiac', 'lpógiac', 'lqógiac', 'lrógiac', 'lsógiac', 'ltógiac', 'luógiac', 'lvógiac', 'lwógiac', 'lxógiac', 'lyógiac', 'lzógiac', 'làógiac', 'láógiac', 'lâógiac', 'lãógiac', 'lèógiac', 'léógiac', 'lêógiac', 'lìógiac', 'líógiac', 'lîógiac', 'lòógiac', 'lóógiac', 'lôógiac', 'lõógiac', 'lùógiac', 'lúógiac', 'lûógiac', 'lçógiac', 'lóagiac', 'lóbgiac', 'lócgiac'

In [68]:
avaliador(lista_teste)

76.34% de 186 palavras


### Descobrindo quantas palavras não conhecemos

Precisamos saber quanto dessa porcentagem de erro (aprox. 24%) está relacionada à palavras desconhecidas

In [70]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
        else:
            desconhecida += (correta not in vocabulario)
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecido = round(desconhecida*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras \ndesconhecida é {taxa_desconhecido}%')

vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

76.34% de 186 palavras 
desconhecida é 6.99%


### Removendo duas letras a mais

In [74]:
palavra = 'lóiigica'

def gerador_turbinado(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras(palavra)
    return novas_palavras

palavras_g = gerador_turbinado(gerador_palavras(palavra))
'lógica' in palavras_g

True

In [75]:
len(palavras_g)

691744

### Excluindo palavras que não existem

In [79]:
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavras_turbinado = gerador_turbinado(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_turbinado)
    candidatos = [palavra]
    for palavra in todas_palavras:
        if palavra in vocabulario:
            candidatos.append(palavra)
    
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

novo_corretor(palavra)

'lógica'

## Avaliando os resultados

In [82]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
        else:
            print(errada + '-' + corretor(errada) + '-' + palavra_corrigida)
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecido = round(desconhecida*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras \ndesconhecida é {taxa_desconhecido}%')

vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

esje-esse-se
sãêo-são-não
dosa-dos-do
eme-em-de
eàssa-essa-esse
daõs-das-da
céda-cada-da
noâ-no-o
enêão-então-não
tĩem-tem-em
nossah-nossa-nosso
teb-tem-de
atĩ-até-a
âem-em-de
foo-foi-o
serr-ser-se
entke-entre-então
van-vai-a
çeus-seus-seu
eû-e-de
temeo-tempo-temos
semre-sempre-ser
elaá-ela-ele
síó-só-se
siàe-site-se
seém-sem-em
peln-pelo-ele
aléra-alura-agora
tdia-dia-da
tuúo-tudo-tipo
jé-é-de
sãô-são-não
odos-dos-do
siua-sua-seu
elpe-ele-esse
teos-temos-os
eũsa-essa-esse
vjmos-vamos-temos
dms-dos-de
cava-java-para
ános-nos-no
èaso-caso-as
túem-tem-em
daáos-dados-dos
nossk-nosso-nosso
tãer-ter-ser
vté-até-é
búm-bem-um
sçerá-será-ser
entró-entre-então
uai-vai-a
sâus-seus-seu
ìeu-seu-de
fual-qual-sua
elal-ela-ele
skó-só-se
secm-sem-em
aluéa-alura-além
dil-dia-de
sód-só-se
eúaa-aeúaa-essa
ró-só-de
dĩaz-adĩaz-da
correptor-corretor-correto
trtica-tática-prática
ewpoderamento-aewpoderamento-ewpoderamento
îgato-gato-fato
cakvalo-acakvalo-carvalho
canelac-acanelac-janela
tênisy-atênisy-tênisy

In [83]:
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecido = round(desconhecida*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras \ndesconhecida é {taxa_desconhecido}%')

vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

76.34% de 186 palavras 
desconhecida é 6.99%


In [85]:
palavra = 'lóiigica'
print(novo_corretor(palavra))
print(corretor(palavra))


lógica
alóiigica
